In [1]:
import glob
import os
import string

import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
import matplotlib.ticker as ticker

from sklearn.metrics import confusion_matrix

### Load Data

The names can be found in text files in a src directory, one file per language.

In the following you can find some utilities to load the data into pandas data frames. 

We will restrict to some common European languages. 

With the given selection, we will identify all the occurring characters and initialize an alphabet.<br>
For this alphabet, we will use a one-hot-encoding to map them into a vector space representation. 

Foresee a suitable character for the end of the word, e.g. 'END'.

In [5]:
srcdir = 'data/names'
languages = ["English","French","Italian","German","Spanish"]

In [6]:
# inspect the data directory
def findFiles(path): 
    return glob.glob(path)

In [7]:
print('\n'.join(findFiles(os.path.join(srcdir,'*.txt'))))

name_classification/data/names/Czech.txt
name_classification/data/names/German.txt
name_classification/data/names/Arabic.txt
name_classification/data/names/Japanese.txt
name_classification/data/names/Chinese.txt
name_classification/data/names/Vietnamese.txt
name_classification/data/names/Russian.txt
name_classification/data/names/French.txt
name_classification/data/names/Irish.txt
name_classification/data/names/English.txt
name_classification/data/names/Spanish.txt
name_classification/data/names/Greek.txt
name_classification/data/names/Italian.txt
name_classification/data/names/Portuguese.txt
name_classification/data/names/Scottish.txt
name_classification/data/names/Dutch.txt
name_classification/data/names/Korean.txt
name_classification/data/names/Polish.txt


In [8]:
# Read a file and split into lines
def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return lines

def load_data(srcdir, categories=None):
    names_list = []
    for filename in findFiles(os.path.join(srcdir,'*.txt')):
        category = os.path.splitext(os.path.basename(filename))[0]
        if not categories or category in categories: 
            names = readLines(filename)
            names_list.extend([(name,category) for name in names])
    df = pd.DataFrame(names_list)
    df.columns = ["name","lang"]
    return df

In [9]:
names = load_data(srcdir,categories=languages)
names.head()

,name,lang
0,Abbing,German
1,Abel,German
2,Abeln,German
3,Abt,German
4,Achilles,German


In [10]:
maxlen = np.max([len(name) for name in names.name])
print("Maximum name length: ", maxlen)

Maximum name length:  18


In [9]:
alphabet = sorted(list(set(''.join([name for name in names.name]))))
alphabet.append('END')
len_alphabet = len(alphabet)
char_index = dict((c, i) for i, c in enumerate(alphabet))
print("Size of alphabet: ",len_alphabet)
print(alphabet)

Size of alphabet:  74
[' ', "'", 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'Á', 'É', 'ß', 'à', 'á', 'ä', 'ç', 'è', 'é', 'ê', 'ì', 'í', 'ñ', 'ò', 'ó', 'ö', 'ù', 'ú', 'ü', 'END']


In [11]:
names.groupby('lang')['name'].count()/len(names)

lang
English    0.646230
French     0.048802
German     0.127555
Italian    0.124912
Spanish    0.052502
Name: name, dtype: float64

### Vector Representations

Now construct the vector representation by using one-hot-vectors. 

In [12]:
language_to_index = {country:index for index,country in enumerate(names.lang.unique())}
index_to_language = {index:country for index,country in enumerate(names.lang.unique())}

def onehot(i, length):
    v = np.zeros(length);
    v[i] = 1
    return v

def name_representation(name, maxlen):
    ### START YOUR CODE
    name_trunc = str(name)[0:maxlen]
    size = len(char_index)
    vector = [onehot(char_index[j], size) for j in str(name)]
    # fill the rest with 
    for k in range(0,maxlen - len(str(name))):
        vector.append(onehot(char_index['END'], size))
    return vector
    ### START YOUR CODE

def lang_representation(language, language_to_index):
    y = np.zeros(len(language_to_index))
    y[language_to_index[language]]=1
    return y

def lang_from_output(score):
    return index_to_language[np.argmax(score)]

def predict(name, model):
    score = model.predict(np.array([name_representation(name, maxlen)]))[0]
    return lang_from_output(score)

### Prepare train/test

Split the data into train/test

Shuffle the data

Transform the names data into a suitable vector respresentation:
* names into numpy arrays of shape (*,maxlen,len_alphabet)
* language into numpy array of shape (*,len(languages))



In [14]:
test_split = 0.2

### START YOUR CODE
# Shuffle and split names data

train = ...

test = ...

### END YOUR CODE

In [ ]:
### START YOUR CODE

# Map train and test data into vector space (one-hot-vectors)

X_train = ...
Y_train = ...

X_test = ...
Y_test = ...

### END YOUR CODE

### Utilities

In [21]:
def performance_plots(model, X_test, Y_test, log, epochs):    
    acc = log.history['accuracy']
    val_acc = log.history['val_accuracy']
    print('Train accuracy (end):', acc[-1])
    print('Train accuracy (max):', np.max(acc))
    print('Test accuracy (end) :', val_acc[-1])
    print('Test accuracy (max) :', np.max(val_acc))

    plt.plot(log.history['accuracy'])
    plt.plot(log.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train','test'], loc='upper left')
    plt.axis([0,epochs,0.0,1.0])
    plt.show()

    plt.plot(log.history['loss'])
    plt.plot(log.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train','test'], loc = 'upper left')
    plt.axis([0,epochs,0.0,2.0])
    plt.show()

In [17]:
import sklearn
import pandas as pd

def confusion_matrix(model, X_test, Y_test):
    scores = model.predict(X_test)
    predictions = np.argmax(scores, axis=1)
    labels = np.argmax(Y_test, axis=1)
    nsamples = Y_test.shape[0]
    cm = sklearn.metrics.confusion_matrix(labels, predictions)
    df = pd.DataFrame({languages[i] : cm[:,i] for i in range(len(languages))}, languages)
    return df, cm

### Define and Train Model: Single Layer with SimpleRNN

Create an RNN consisting of a single layer with a SimpleRNN (keras) and a softmax.

Then train the model. Play with different number of hidden units in the layer to obtain a good accuracy.

In [11]:
### START YOUR CODE

# SImpleRNN, single layer with tf.keras....

model = ...

model.summary()

### END YOUR CODE

In [19]:
### START YOUR CODE
batch_size=
nepochs = 

log = model.fit(X_train, Y_train, batch_size=batch_size, \
                nb_epoch=nepochs, validation_data=(X_test, Y_test))

### END YOUR CODE

Train on 4545 samples, validate on 1131 samples
Epoch 1/10
4545/4545 [==============================] - 3s 758us/sample - loss: 1.1283 - accuracy: 0.6345 - val_loss: 1.0387 - val_accuracy: 0.6525
Epoch 2/10
4545/4545 [==============================] - 2s 340us/sample - loss: 0.9673 - accuracy: 0.6693 - val_loss: 0.9022 - val_accuracy: 0.6941
Epoch 3/10
4545/4545 [==============================] - 1s 322us/sample - loss: 0.8295 - accuracy: 0.7043 - val_loss: 0.8138 - val_accuracy: 0.7135
Epoch 4/10
4545/4545 [==============================] - 2s 382us/sample - loss: 0.7588 - accuracy: 0.7274 - val_loss: 0.7944 - val_accuracy: 0.7197
Epoch 5/10
4545/4545 [==============================] - 2s 358us/sample - loss: 0.7347 - accuracy: 0.7360 - val_loss: 0.7714 - val_accuracy: 0.7383
Epoch 6/10
4545/4545 [==============================] - 2s 401us/sample - loss: 0.6700 - accuracy: 0.7613 - val_loss: 0.7087 - val_accuracy: 0.7480
Epoch 7/10
4545/4545 [==============================] - 2s 398us

In [ ]:
performance_plots(model, X_test, Y_test, log, nepochs)
df, cm = confusion_matrix(model, X_test, Y_test)
df

### Handle Class Imbalance, Resampling

We can observe a quite significant class imbalance in the data.<br>

One way to compensate for that would be to use resampling.

In [26]:
def handle_imbalance(names, languages):

    ### START YOUR CODE

    # treat class imbalance
    names_upsampled = 

    ### END YOUR CODE    
    return names_upsampled


In [27]:
names_upsampled = handle_imbalance(names, languages)
names_upsampled.groupby('lang')['name'].count()/len(names_upsampled)

lang
English    0.2
French     0.2
German     0.2
Italian    0.2
Spanish    0.2
Name: name, dtype: float64

#### Train Model

Again prepare train and test data (X_train, X_test, Y_train, Y_test)

Train the same model as above with the new data.

In [ ]:
### START YOUR CODE

X_train = ...
X_test = ...

Y_train = ...
Y_test = ...

### END YOUR CODE

In [ ]:
### START YOUR CODE

# train model

...

### END YOUR CODE

#### Findings

...


#### In which situations is class imbalance treatment important and why?  

....

### Implement Model with several SimpleRNN Layers

In [ ]:
### START YOUR CODE

model = ...

### END YOUR CODE

model.summary()

In [ ]:
### START YOUR CODE

# train...

### END YOUR CODE

#### Findings

...
